In [1]:
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch import optim

from dataloader import make_train_df, SIIMData, ImageTransform
from trainer import Trainer
from classification_model import Model
%matplotlib inline

In [2]:

size = (400, 400)
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.2)
train = make_train_df()[:2]
train_df, val_df = train_test_split(train, test_size=0.25)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

train_dataset = SIIMData(df=train_df, transform=ImageTransform(size, mean, std), phase="train")
val_dataset = SIIMData(df=val_df, transform=ImageTransform(size, mean, std), phase="val")

  0%|          | 0/6334 [00:00<?, ?it/s]

In [3]:
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

C:\Users\ryuhe\Anaconda3\envs\KAGGLE\lib\site-packages\torch\utils\data\dataloader.py:474: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
model = Model(model_name="tf_efficientnet_l2_ns", pretrained=True)
params_to_update_0 = []
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []
params_to_update_4 = []
params_to_update_5 = []
params_to_update_6 = []
params_to_update_7 = []

update_param_name_0 = "model.blocks.0"
update_param_name_1 = "model.blocks.1"
update_param_name_2 = "model.blocks.2"
update_param_name_3 = "model.blocks.3"
update_param_name_4 = "model.blocks.4"
update_param_name_5 = "model.blocks.5"
update_param_name_6 = "model.blocks.6"
update_param_name_7 = "classifier"

for name, param in model.named_parameters():
    if update_param_name_0 in name:
        param.requires_grad = True
        params_to_update_0.append(param)
    elif update_param_name_1 in name:
        param.requires_grad = True
        params_to_update_1.append(param)
    elif update_param_name_2 in name:
        param.requires_grad = True
        params_to_update_2.append(param)
    elif update_param_name_3 in name:
        param.requires_grad = True
        params_to_update_3.append(param)
    elif update_param_name_4 in name:
        param.requires_grad = True
        params_to_update_4.append(param)
    elif update_param_name_5 in name:
        param.requires_grad = True
        params_to_update_5.append(param)
    elif update_param_name_6 in name:
        param.requires_grad = True
        params_to_update_6.append(param)
    elif update_param_name_7 in name:
        param.requires_grad = True
        params_to_update_7.append(param)
    else:
        param.requires_grad = False

optimizer = optim.Adam([
    {"params": params_to_update_0, "lr": 1e-4},
    {"params": params_to_update_1, "lr": 2e-4},
    {"params": params_to_update_2, "lr": 3e-4},
    {"params": params_to_update_3, "lr": 5e-4},
    {"params": params_to_update_4, "lr": 7e-4},
    {"params": params_to_update_5, "lr": 8e-4},
    {"params": params_to_update_6, "lr": 9e-4},
    {"params": params_to_update_7, "lr": 1e-3},
])

criterion = nn.CrossEntropyLoss()

trainer = Trainer(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    model=model,
    criterion=criterion,
    optimizer=optimizer)


In [5]:
num_epochs = 3

train_loss_list = []
train_acc_list = []
valid_loss_list = []
valid_acc_list = []

no_train_loss, no_train_acc = trainer.valid_one_cycle()
print(f"No_train Loss: {no_train_loss} Acc: {no_train_acc}")


for epoch in range(1, num_epochs+1):
    torch.backends.cudnn.benchmark = True
    print(f"Epoch {epoch}/{num_epochs}")
    print("-" * 20)
    epoch_train_loss, epoch_train_accuracy = trainer.train_one_cycle()
    train_loss_list.append(epoch_train_loss)
    train_acc_list.append(epoch_train_accuracy)
    print(f"train Loss: {epoch_train_loss} Acc: {epoch_train_accuracy}")
    epoch_valid_loss, epoch_valid_accuracy = trainer.valid_one_cycle()
    valid_loss_list.append(epoch_valid_loss)
    valid_acc_list.append(epoch_valid_accuracy)
    print(f"valid Loss: {epoch_valid_loss} Acc: {epoch_valid_accuracy}")

  0%|          | 0/1 [00:00<?, ?it/s]

No_train Loss: 1.247510552406311 Acc: 1.0
Epoch 1/3
--------------------


  0%|          | 0/1 [00:00<?, ?it/s]

train Loss: 1.575831413269043 Acc: 0.0


  0%|          | 0/1 [00:00<?, ?it/s]

valid Loss: 1.9913922548294067 Acc: 0.0
Epoch 2/3
--------------------


  0%|          | 0/1 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000002799C7F7040>
Traceback (most recent call last):
  File "C:\Users\ryuhe\Anaconda3\envs\KAGGLE\lib\site-packages\torch\utils\data\dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "C:\Users\ryuhe\Anaconda3\envs\KAGGLE\lib\site-packages\torch\utils\data\dataloader.py", line 1292, in _shutdown_workers
    self._mark_worker_as_unavailable(worker_id, shutdown=True)
  File "C:\Users\ryuhe\Anaconda3\envs\KAGGLE\lib\site-packages\torch\utils\data\dataloader.py", line 1239, in _mark_worker_as_unavailable
    q.put(None)
  File "C:\Users\ryuhe\Anaconda3\envs\KAGGLE\lib\multiprocessing\queues.py", line 88, in put
    self._start_thread()
  File "C:\Users\ryuhe\Anaconda3\envs\KAGGLE\lib\multiprocessing\queues.py", line 173, in _start_thread
    self._thread.start()
  File "C:\Users\ryuhe\Anaconda3\envs\KAGGLE\lib\threading.py", line 852, in start
    _start_new_thread(self._bootstrap, ())
Runtim

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:75] data. DefaultCPUAllocator: not enough memory: you tried to allocate 16900 bytes. Buy new RAM!